In [321]:
import pandas as pd
import numpy as np

from random import randrange

from sklearn.metrics import confusion_matrix, classification_report

# Data Training 

In [322]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
df = pd.read_csv(r"C:\Users\Allan\Desktop\Artificial-Neural-Network\Single-Layer-Perceptron\data\train_data_cl.csv", header=None, names=list(range(13)))

df.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,NaN
1,1,0,3,Braund,Mr. Owen Harris,male,22,1,0,A/5 21171,7.25,NaN,S
2,2,1,1,Cumings,Mrs. John Bradley (Florence Briggs Thayer),female,38,1,0,PC 17599,712.833,C85,C
3,3,1,3,Heikkinen,Miss. Laina,female,26,0,0,STON/O2. 3101282,7.925,NaN,S
4,4,1,1,Futrelle,Mrs. Jacques Heath (Lily May Peel),female,35,1,0,113803,53.1,C123,S


### Data Preprocessing

In [323]:
### Benerin column
col_name = df.iloc[0,:12].values
col_name = np.insert(col_name, 4, "FirstName")

df.columns = col_name
df = df.drop(0).reset_index(drop=True)

df.head()
### Merge Name dan FirstName
df["Name"] = df["FirstName"] + " " + df["Name"]
df = df.drop("FirstName", axis=1)
df
### Perbaikan data yang tidak sesuai

print(df['Sex'].unique().tolist())
true_sex_val = ['male', 'female']
count = 0
for val in df["Sex"].values:
    if val not in true_sex_val:
        df.loc[count, 'Sex'] = np.random.choice(true_sex_val)
    count += 1
print(df['Sex'].unique().tolist())

count = 0
for val in df['Fare'].values:
    try:
        float(val)
    except ValueError:
        df.loc[count, 'Fare'] = 0
    count += 1

count = 0
for val in df['Parch'].values:
    if len(val) > 1:
        df.loc[count, 'Parch'] = 0
    count += 1

['male', 'female', '15', '27', '9', '36.5', '16', '40', '45', '24', '18', '20.5', '8', '26', '3', '1', '19', '21', '36', '22', '48', '49', '35', '39', '6']
['male', 'female']


In [324]:
### Encode
lbenc = LabelEncoder()

for i in df.columns.values:
    if df[i].dtypes == 'string':
        df[i] = lbenc.fit_transform(df[i])

df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,Mr. Owen Harris Braund,male,22,1,0,A/5 21171,7.25,NaN,S
1,2,1,1,Mrs. John Bradley (Florence Briggs Thayer) Cu...,female,38,1,0,PC 17599,712.833,C85,C
2,3,1,3,Miss. Laina Heikkinen,female,26,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,Mrs. Jacques Heath (Lily May Peel) Futrelle,female,35,1,0,113803,53.1,C123,S
4,5,0,3,Mr. William Henry Allen,male,35,0,0,373450,8.05,NaN,S


In [325]:
### Change Data Type
df.info()
df = df.astype({'PassengerId': 'int64', 'Survived': 'int64', 'Pclass': 'int64', 'SibSp': 'int64', 'Parch': 'int64'})
df = df.astype({'Name': 'string', 'Sex': 'string', 'Ticket': 'string', 'Cabin': 'string', 'Embarked': 'string'})
df = df.astype({'Age': 'float64', 'Fare': 'float64'})
df.info()
### Menghilangkan Column yang tidak dipakai
del df['Name']
del df['Ticket']
del df['Cabin']
### Cek & Replace Missing Value
missing = pd.DataFrame({
    'total' : df.isnull().sum(),
    'percent' : df.isnull().sum()/df.shape[0] * 100})

missing
df['Embarked'] = df['Embarked'].fillna(value=np.random.choice(df['Embarked'].unique().tolist()))
df['Fare'] = df['Fare'].fillna(value=0)
df.isna().sum()
clean_data = df.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  712 non-null    object
 1   Survived     712 non-null    object
 2   Pclass       712 non-null    object
 3   Name         712 non-null    object
 4   Sex          712 non-null    object
 5   Age          712 non-null    object
 6   SibSp        712 non-null    object
 7   Parch        712 non-null    object
 8   Ticket       712 non-null    object
 9   Fare         685 non-null    object
 10  Cabin        210 non-null    object
 11  Embarked     677 non-null    object
dtypes: object(12)
memory usage: 66.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 712 entries, 0 to 711
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Survived     712 non-null    int64  
 2   P

In [326]:
## Encode
lbenc = LabelEncoder()

for i in df.columns.values:
    if df[i].dtypes == 'string':
        df[i] = lbenc.fit_transform(df[i])

df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.250,2
1,2,1,1,0,38.0,1,0,712.833,0
2,3,1,3,0,26.0,0,0,7.925,2
3,4,1,1,0,35.0,1,0,53.100,2
4,5,0,3,1,35.0,0,0,8.050,2


In [327]:
### Split Feature and Label
train = df.copy()
x_train = train.iloc[:, 2:].values
y_train = train.iloc[:, 1].values
train

### Normalisasi

scaler = MinMaxScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns.astype(str))
df_normalized

### Split Feature and Label
train_normalized = df_normalized.copy()
x_train_normalized = train_normalized.iloc[:, 2:].values
y_train_normalized = train_normalized.iloc[:, 1].values
train_normalized = pd.DataFrame(train_normalized, columns=train_normalized.columns)

id_column = df['PassengerId']
train_normalized = train_normalized.drop(columns=['PassengerId'])
train_normalized.insert(0, 'PassengerId', id_column)
train_normalized

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0.0,1.0,1.0,0.2750,0.2,0.000000,0.007960,1.0
1,2,1.0,0.0,0.0,0.4750,0.2,0.000000,0.782652,0.0
2,3,1.0,1.0,0.0,0.3250,0.0,0.000000,0.008701,1.0
3,4,1.0,0.0,0.0,0.4375,0.2,0.000000,0.058301,1.0
4,5,0.0,1.0,1.0,0.4375,0.0,0.000000,0.008838,1.0
...,...,...,...,...,...,...,...,...,...
707,886,0.0,1.0,0.0,0.4875,0.0,0.833333,0.031978,0.5
708,887,0.0,0.5,1.0,0.3375,0.0,0.000000,0.014273,1.0
709,888,1.0,0.0,0.0,0.2375,0.0,0.000000,0.032938,1.0
710,890,1.0,0.0,1.0,0.3250,0.0,0.000000,0.032938,0.0


# Data Training v2

In [328]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
df_v2 = pd.read_csv(r"C:\Users\Allan\Desktop\Artificial-Neural-Network\Single-Layer-Perceptron\data\train_data_cl_v2.csv", header=None, names=list(range(13)))

# Assuming your DataFrame is named df
df_v2.columns = df_v2.iloc[0]
df_v2 = df_v2[1:]

df_v2 = df_v2.dropna(axis=1)


df_v2

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
1,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,S
2,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C
3,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,S
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,S
5,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,S
...,...,...,...,...,...,...,...,...,...,...,...
708,886,0,3,"Rice, Mrs. William (Margaret Norton)",female,39,0,5,382652,29.125,Q
709,887,0,2,"Montvila, Rev. Juozas",male,27,0,0,211536,13,S
710,888,1,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30,S
711,890,1,1,"Behr, Mr. Karl Howell",male,26,0,0,111369,30,C


### Data Preprocessing

In [329]:
# Assuming your DataFrame is named df
columns_to_drop = ['Name', 'Ticket']
df_v2 = df_v2.drop(columns=columns_to_drop)

### Encode
lbenc_v2 = LabelEncoder()

for i in df_v2.columns.values:
    if df_v2[i].dtype == 'O':  # Check for object type
        df_v2[i] = lbenc_v2.fit_transform(df_v2[i])

df_v2

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
1,0,0,2,1,21,1,0,157,2
2,91,1,0,0,45,1,0,175,0
3,182,1,2,0,27,0,0,173,2
4,269,1,0,0,41,1,0,128,2
5,351,0,2,1,41,0,0,187,2
...,...,...,...,...,...,...,...,...,...
708,696,0,2,0,46,0,5,90,1
709,697,0,1,1,28,0,0,21,2
710,698,1,0,0,16,0,0,92,2
711,700,1,0,1,27,0,0,92,0


In [330]:
### Split Feature and Label
train_v2 = df_v2.copy()
x_train_v2 = train_v2.iloc[:, 2:].values
y_train_v2 = train_v2.iloc[:, 1].values
train_v2

### Normalisasi
scaler = MinMaxScaler()
df_normalized_v2 = pd.DataFrame(scaler.fit_transform(df_v2), columns=df_v2.columns.astype(str))
df_normalized_v2

### Split Feature and Label
train_normalized_v2 = df_normalized_v2.copy()
x_train_normalized_v2 = train_normalized_v2.iloc[:, 2:].values
y_train_normalized_v2 = train_normalized_v2.iloc[:, 1].values



id_column_v2 = df['PassengerId']
train_normalized_v2 = train_normalized_v2.drop(columns=['PassengerId'])
train_normalized_v2.insert(0, 'PassengerId', id_column_v2)
train_normalized_v2

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0.0,1.0,1.0,0.241379,0.2,0.000000,0.720183,1.0
1,2,1.0,0.0,0.0,0.517241,0.2,0.000000,0.802752,0.0
2,3,1.0,1.0,0.0,0.310345,0.0,0.000000,0.793578,1.0
3,4,1.0,0.0,0.0,0.471264,0.2,0.000000,0.587156,1.0
4,5,0.0,1.0,1.0,0.471264,0.0,0.000000,0.857798,1.0
...,...,...,...,...,...,...,...,...,...
707,886,0.0,1.0,0.0,0.528736,0.0,0.833333,0.412844,0.5
708,887,0.0,0.5,1.0,0.321839,0.0,0.000000,0.096330,1.0
709,888,1.0,0.0,0.0,0.183908,0.0,0.000000,0.422018,1.0
710,890,1.0,0.0,1.0,0.310345,0.0,0.000000,0.422018,0.0


# Data Testing

In [331]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

dt = pd.read_csv(
    r"C:\Users\Allan\Desktop\Artificial-Neural-Network\Single-Layer-Perceptron\data\test_data_cl.csv",
    header=None,
    names=list(range(12)),
)

gdt = pd.read_csv(
    r"C:\Users\Allan\Desktop\Artificial-Neural-Network\Single-Layer-Perceptron\data\test_data_GroundTruth_cl.csv"
)
dt = dt.head(331)
dt

col_name = dt.iloc[0, :11].values
col_name = np.insert(col_name, 3, "FirstName")

dt.columns = col_name
dt = dt.drop(0).reset_index(drop=True)
dt

dt["Name"] = dt["FirstName"] + " " + dt["Name"]
dt = dt.drop("FirstName", axis=1)
dt

print(dt["Sex"].unique().tolist())
true_sex_val = ["male", "female"]
count = 0
for val in dt["Sex"].values:
    if val not in true_sex_val:
        dt.loc[count, "Sex"] = np.random.choice(true_sex_val)
    count += 1
print(dt["Sex"].unique().tolist())

count = 0
for val in dt["Fare"].values:
    try:
        float(val)
    except ValueError:
        dt.loc[count, "Fare"] = 0
    count += 1

count = 0
for val in dt["Parch"].values:
    if len(val) > 1:
        dt.loc[count, "Parch"] = 0
    count += 1

dt.info()

dt = dt.astype(
    {"PassengerId": "int64", "Pclass": "int64", "SibSp": "int64", "Parch": "int64"}
)
dt = dt.astype(
    {
        "Name": "string",
        "Sex": "string",
        "Ticket": "string",
        "Cabin": "string",
        "Embarked": "string",
    }
)
dt = dt.astype({"Age": "float64", "Fare": "float64"})
dt.info()

del dt["Name"]
del dt["Ticket"]
del dt["Cabin"]

missing = pd.DataFrame(
    {"total": dt.isnull().sum(), "percent": dt.isnull().sum() / dt.shape[0] * 100}
)

missing

dt["Embarked"] = dt["Embarked"].fillna(
    value=np.random.choice(dt["Embarked"].unique().tolist())
)
dt["Fare"] = dt["Fare"].fillna(value=0)
dt.isna().sum()

['male', 'female', '45', '18.5', '36', '20', '16', '42', '29', '30', '46', '19', '0.17', '22', '21']
['male', 'female']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  330 non-null    object
 1   Pclass       330 non-null    object
 2   Name         330 non-null    object
 3   Sex          330 non-null    object
 4   Age          330 non-null    object
 5   SibSp        330 non-null    object
 6   Parch        330 non-null    object
 7   Ticket       330 non-null    object
 8   Fare         317 non-null    object
 9   Cabin        100 non-null    object
 10  Embarked     316 non-null    object
dtypes: object(11)
memory usage: 28.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Passen

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [332]:
lbenc = LabelEncoder()

for col in dt.columns.values:
    if dt[col].dtype == "string":
        dt[col] = lbenc.fit_transform(dt[col])

dt.head()

merged_test_data = pd.merge(dt, gdt, on="PassengerId")
merged_test_data

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,892,3,1,34.5,0,0,78.292,1,0
1,893,3,0,47.0,1,0,7.000,2,1
2,894,2,1,62.0,0,0,96.875,1,0
3,895,3,1,27.0,0,0,86.625,2,0
4,896,3,0,22.0,1,1,122.875,2,1
...,...,...,...,...,...,...,...,...,...
325,1299,1,1,50.0,1,1,211.500,0,0
326,1301,3,0,3.0,1,1,13.775,2,1
327,1303,1,0,37.0,1,0,90.000,1,1
328,1304,3,0,28.0,0,0,7.775,2,1


In [333]:
test = merged_test_data.copy()

X_test = test.iloc[:, 1:8]
y_test = test.iloc[:, 8]

feature = X_test.copy()
label = y_test.copy().values.reshape(-1, 1)
passengerId = test.iloc[:, 0].values.reshape(-1, 1)

col = test.columns.values.tolist()
col.pop(8) 

mnmx = MinMaxScaler()
feature = mnmx.fit_transform(feature)

data = np.concatenate((passengerId, feature), axis=1)
normalize_data_test = pd.DataFrame(data, columns=col)

normalize_data_test['Survived'] = y_test.values

normalize_data_test.drop('PassengerId', axis=1, inplace=True)
normalize_data_test = normalize_data_test.astype({"Survived": "int64"})

X_test_normalized = normalize_data_test.iloc[:, :-1].values
y_test = normalize_data_test.iloc[:, -1].values 

normalize_data_test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,1.0,1.0,0.453947,0.000,0.000000,0.094148,0.5,0
1,1.0,0.0,0.618421,0.125,0.000000,0.008418,1.0,1
2,0.5,1.0,0.815789,0.000,0.000000,0.116495,0.5,0
3,1.0,1.0,0.355263,0.000,0.000000,0.104169,1.0,0
4,1.0,0.0,0.289474,0.125,0.166667,0.147760,1.0,1
...,...,...,...,...,...,...,...,...
325,0.0,1.0,0.657895,0.125,0.166667,0.254334,0.0,0
326,1.0,0.0,0.039474,0.125,0.166667,0.016565,1.0,1
327,0.0,0.0,0.486842,0.125,0.000000,0.108227,0.5,1
328,1.0,0.0,0.368421,0.000,0.000000,0.009350,1.0,1


In [334]:
dimensi = f'dimensi x_train\t: {x_train.shape}\ndimensi x_test\t: {X_test_normalized.shape}\ndimensi y_train\t: {y_train.shape}\ndimensi y_test\t: {y_test.shape}\n'
print(dimensi)

dimensi x_train	: (712, 7)
dimensi x_test	: (330, 7)
dimensi y_train	: (712,)
dimensi y_test	: (330,)



# Data Testing v2

In [335]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder, MinMaxScaler

dt_v2 = pd.read_csv(
    r"C:\Users\Allan\Desktop\Artificial-Neural-Network\Single-Layer-Perceptron\data\test_data_cl_v2.csv",
    header=None,
    names=list(range(12)),
)

# Assuming your DataFrame is named df
dt_v2.columns = dt_v2.iloc[0]
dt_v2 = dt_v2[1:]

dt_v2 = dt_v2.dropna(axis=1)

gdt_v2 = pd.read_csv(
    r"C:\Users\Allan\Desktop\Artificial-Neural-Network\Single-Layer-Perceptron\data\test_data_GroundTruth_cl.csv"
)
dt_v2 = dt_v2.head(331)

dt_v2


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
1,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q
2,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47,1,0,363272,7,S
3,894,2,"Myles, Mr. Thomas Francis",male,62,0,0,240276,9.6875,Q
4,895,3,"Wirz, Mr. Albert",male,27,0,0,315154,8.6625,S
5,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22,1,1,3101298,12.2875,S
...,...,...,...,...,...,...,...,...,...,...
327,1301,3,"Peacock, Miss. Treasteall",female,3,1,1,SOTON/O.Q. 3101315,13.775,S
328,1303,1,"Minahan, Mrs. William Edward (Lillian E Thorpe)",female,37,1,0,19928,90,Q
329,1304,3,"Henriksson, Miss. Jenny Lovisa",female,28,0,0,347086,7.775,S
330,1306,1,"Oliva y Ocana, Dona. Fermina",female,39,0,0,PC 17758,108.9,C


In [336]:
dt_v2 = dt_v2.astype(
    {"PassengerId": "int64", "Pclass": "int64", "SibSp": "int64", "Parch": "int64"}
)
dt_v2 = dt_v2.astype(
    {
        "Name": "string",
        "Sex": "string",
        "Ticket": "string",
        "Embarked": "string",
    }
)

# Assuming your DataFrame is named df
columns_to_drop = ['Name', 'Ticket']
dt_v2 = dt_v2.drop(columns=columns_to_drop)

### Encode
lbenc_v2 = LabelEncoder()

for i in dt_v2.columns.values:
    if dt_v2[i].dtype == 'string':  # Check for object type
        dt_v2[i] = lbenc_v2.fit_transform(dt_v2[i])

print("dt_v2 Info:")
print(dt_v2.info())

print("\ngdt_v2 Info:")
print(gdt_v2.info())

merged_test_data_v2 = pd.merge(dt_v2, gdt_v2, on="PassengerId")
merged_test_data_v2

dt_v2 Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331 entries, 1 to 331
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   PassengerId  331 non-null    int64 
 1   Pclass       331 non-null    int64 
 2   Sex          331 non-null    int32 
 3   Age          331 non-null    object
 4   SibSp        331 non-null    int64 
 5   Parch        331 non-null    int64 
 6   Fare         331 non-null    object
 7   Embarked     331 non-null    int32 
dtypes: int32(2), int64(4), object(2)
memory usage: 18.2+ KB
None

gdt_v2 Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331 entries, 0 to 330
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   PassengerId  331 non-null    int64
 1   Survived     331 non-null    int64
dtypes: int64(2)
memory usage: 5.3 KB
None


,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,892,3,1,34.5,0,0,7.8292,1,0
1,893,3,0,47,1,0,7,2,1
2,894,2,1,62,0,0,9.6875,1,0
3,895,3,1,27,0,0,8.6625,2,0
4,896,3,0,22,1,1,12.2875,2,1
...,...,...,...,...,...,...,...,...,...
326,1301,3,0,3,1,1,13.775,2,1
327,1303,1,0,37,1,0,90,1,1
328,1304,3,0,28,0,0,7.775,2,1
329,1306,1,0,39,0,0,108.9,0,1


In [337]:
# Copy Data
test_v2 = merged_test_data_v2.copy()
test_v2

# Separate Features and Labels
X_test_v2 = test_v2.iloc[:, 1:8]
y_test_v2 = test_v2.iloc[:, 8]

# Extract Passenger IDs
passengerId_v2 = test_v2['PassengerId'].values.reshape(-1, 1)

# Remove Label from Columns
col_v2 = test_v2.columns.values.tolist()
col_v2.pop(8)

# MinMax Scaling
mnmx_v2 = MinMaxScaler()
X_test_normalized_v2 = mnmx_v2.fit_transform(X_test_v2)

# Create DataFrame
normalize_data_test_v2 = pd.DataFrame(np.concatenate((passengerId_v2, X_test_normalized_v2), axis=1), columns=col)

# Include Labels
normalize_data_test_v2['Survived'] = y_test_v2.astype(int)

# Drop PassengerId
normalize_data_test_v2.drop('PassengerId', axis=1, inplace=True)

normalize_data_test_v2


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,1.0,1.0,0.452723,0.000,0.000000,0.015282,0.5,0
1,1.0,0.0,0.617566,0.125,0.000000,0.013663,1.0,1
2,0.5,1.0,0.815377,0.000,0.000000,0.018909,0.5,0
3,1.0,1.0,0.353818,0.000,0.000000,0.016908,1.0,0
4,1.0,0.0,0.287881,0.125,0.166667,0.023984,1.0,1
...,...,...,...,...,...,...,...,...
326,1.0,0.0,0.037320,0.125,0.166667,0.026887,1.0,1
327,0.0,0.0,0.485692,0.125,0.000000,0.175668,0.5,1
328,1.0,0.0,0.367005,0.000,0.000000,0.015176,1.0,1
329,0.0,0.0,0.512066,0.000,0.000000,0.212559,0.0,1


# Model 1

### Function

In [352]:
class SingleLayerPerceptron:
    def __init__(self, input_size, learning_rate, epochs):
        self.weights = np.zeros(input_size + 1)  # Additional weight for bias
        self.learning_rate = learning_rate
        self.epochs = epochs

    def _activation_function(self, x):
        # Using a simple step function as the activation function
        return 1 if x >= 0 else 0

    def _predict(self, inputs):
        summation = np.dot(inputs, self.weights[1:]) + self.weights[0]
        return self._activation_function(summation)

    def _calculate_accuracy(self, X, y):
        predictions = self.predict(X)
        accuracy = np.mean(predictions == y)
        return accuracy * 100

    def train(self, X_train, y_train, X_test, y_test):
        for epoch in range(self.epochs):
            for inputs, label in zip(X_train, y_train):
                prediction = self._predict(inputs)
                error = label - prediction

                # Update weights
                self.weights[1:] += self.learning_rate * error * inputs
                self.weights[0] += self.learning_rate * error

            # Calculate training accuracy at each epoch
            training_accuracy = self._calculate_accuracy(X_train, y_train)
            # print(f"Epoch {epoch + 1}/{self.epochs}: Training Accuracy = {training_accuracy:.2f}%")

        # Print final training accuracy
        final_training_accuracy = self._calculate_accuracy(X_train, y_train)
        print(f"\nFinal Training Accuracy = {final_training_accuracy:.2f}%")

        # Print test accuracy
        test_accuracy = self._calculate_accuracy(X_test, y_test)
        print(f"Test Accuracy = {test_accuracy:.2f}%")

    def predict(self, X_test):
        predictions = [self._predict(inputs) for inputs in X_test]
        return np.array(predictions)

### Use (lr 0.1)

In [360]:
epoch = 100
lr = 0.1

input_size = x_train.shape[1]
perceptron = SingleLayerPerceptron(input_size, lr, epoch)
perceptron.train(x_train, y_train, X_test_normalized, y_test)


Final Training Accuracy = 62.64%
Test Accuracy = 98.18%


In [361]:
predictions = perceptron.predict(X_test_normalized)
print(predictions)

[0 1 0 0 1 0 1 0 1 0 0 1 0 1 1 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 0 1 1 0 0 0 1
 1 0 0 1 1 0 0 1 1 0 0 0 1 0 0 0 1 0 1 0 0 1 1 0 1 0 1 0 1 0 1 0 0 0 1 1 0
 1 1 0 0 1 0 1 0 1 0 0 1 0 0 0 0 1 1 1 0 1 0 1 1 1 0 1 0 0 0 0 0 0 0 0 0 1
 0 1 1 0 0 0 0 0 0 1 1 0 0 1 1 0 1 0 1 0 1 0 0 1 0 0 0 1 1 0 1 1 0 0 1 1 0
 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 1 1 0 1 0 1 0 0 0 0 1 0
 0 0 0 0 1 1 1 1 0 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 1 0 1
 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0
 1 0 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 0 0 1 0 0 1
 1 1 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 1 1 1 1]


### Use (lr 0.0001)

In [363]:
epoch = 100
lr = 0.0001

input_size = x_train.shape[1]
perceptron = SingleLayerPerceptron(input_size, lr, epoch)
perceptron.train(x_train, y_train, X_test_normalized, y_test)


Final Training Accuracy = 62.64%
Test Accuracy = 98.18%


### Use 2 (lr 0.1)

In [364]:
epoch = 1000
lr = 0.1

input_size = x_train_v2.shape[1]
perceptron = SingleLayerPerceptron(input_size, lr, epoch)
perceptron.train(x_train_v2, y_train_v2, X_test_normalized_v2, y_test_v2)


Final Training Accuracy = 78.09%
Test Accuracy = 90.63%


### Use 2 (lr 0.0001)

In [366]:
epoch = 1000
lr = 0.0001

input_size = x_train_v2.shape[1]
perceptron = SingleLayerPerceptron(input_size, lr, epoch)
perceptron.train(x_train_v2, y_train_v2, X_test_normalized_v2, y_test_v2)


Final Training Accuracy = 77.39%
Test Accuracy = 90.63%


# Model 2

### Function 2

In [369]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluate an algorithm without cross-validation(?) ato pake cross-validation ya?
def evaluate_algorithm(train_set, test_set, algorithm, *args):
    actual_train = [row[-1] for row in train_set]
    predicted_train = algorithm(train_set, train_set, *args)
    accuracy_train = accuracy_metric(actual_train, predicted_train)

    actual_test = [row[-1] for row in test_set]
    predicted_test = algorithm(train_set, test_set, *args)
    accuracy_test = accuracy_metric(actual_test, predicted_test)

    return accuracy_train, accuracy_test

# Make a prediction with weights
def predict(row, weights):
    activation = weights[0]
    for i in range(len(row)-1):
        activation += weights[i + 1] * row[i]
    return 1.0 if activation >= 0.0 else 0.0

# Estimate Perceptron weights using stochastic gradient descent
def train_weights(train, l_rate, n_epoch):
    weights = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        for row in train:
            prediction = predict(row, weights)
            error = row[-1] - prediction
            weights[0] = weights[0] + l_rate * error
            for i in range(len(row)-1):
                weights[i + 1] = weights[i + 1] + l_rate * error * row[i]
        
        # Print accuracy after each epoch
        train_predictions = [predict(row, weights) for row in train]
        train_actual = [row[-1] for row in train]
        train_accuracy = accuracy_metric(train_actual, train_predictions)
        # print(f'Epoch {epoch + 1}/{n_epoch}: Training Accuracy = {train_accuracy:.2f}%')
    
    return weights

# Perceptron Algorithm With Stochastic Gradient Descent
def perceptron(train, test, l_rate, n_epoch):
    predictions = list()
    weights = train_weights(train, l_rate, n_epoch)
    for row in test:
        prediction = predict(row, weights)
        predictions.append(prediction)
    return predictions

### Use 1 (lr 0.1)

In [370]:
train_dataset = [list(x_train[i]) + [y_train[i]] for i in range(len(x_train))]

# Combine X_test and y_test into a test dataset
test_dataset = [list(X_test_normalized[i]) + [y_test[i]] for i in range(len(X_test_normalized))]

# Set the learning rate and number of epochs
learning_rate = 0.1
epochs = 100

# # Example: Using the perceptron algorithm without cross-validation
accuracy_train, accuracy_test = evaluate_algorithm(train_dataset, test_dataset, perceptron, learning_rate, epochs)

# # Print the accuracy scores
print(f'Training Accuracy: {accuracy_train:.2f}%')
print(f'Testing Accuracy: {accuracy_test:.2f}%')

Training Accuracy: 62.64%
Testing Accuracy: 98.18%


### Use 1 (lr 0.0001)

In [372]:
train_dataset = [list(x_train[i]) + [y_train[i]] for i in range(len(x_train))]

# Combine X_test and y_test into a test dataset
test_dataset = [list(X_test_normalized[i]) + [y_test[i]] for i in range(len(X_test_normalized))]

# Set the learning rate and number of epochs
learning_rate = 0.0001
epochs = 100

# # Example: Using the perceptron algorithm without cross-validation
accuracy_train, accuracy_test = evaluate_algorithm(train_dataset, test_dataset, perceptron, learning_rate, epochs)

# # Print the accuracy scores
print(f'Training Accuracy: {accuracy_train:.2f}%')
print(f'Testing Accuracy: {accuracy_test:.2f}%')

Training Accuracy: 62.64%
Testing Accuracy: 98.18%


### Use 2 (lr 0.1)

In [374]:
train_dataset = [list(x_train_v2[i]) + [y_train_v2[i]] for i in range(len(x_train_v2))]

# Combine X_test and y_test into a test dataset
test_dataset = [list(X_test_normalized_v2[i]) + [y_test_v2[i]] for i in range(len(X_test_normalized_v2))]

# Set the learning rate and number of epochs
learning_rate = 0.1
epochs = 1000

# # Example: Using the perceptron algorithm without cross-validation
accuracy_train, accuracy_test = evaluate_algorithm(train_dataset, test_dataset, perceptron, learning_rate, epochs)

# # Print the accuracy scores
print(f'Training Accuracy: {accuracy_train:.2f}%')
print(f'Testing Accuracy: {accuracy_test:.2f}%')

Training Accuracy: 78.09%
Testing Accuracy: 90.63%


### Use 2 (lr 0.0001)

In [375]:
train_dataset = [list(x_train_v2[i]) + [y_train_v2[i]] for i in range(len(x_train_v2))]

# Combine X_test and y_test into a test dataset
test_dataset = [list(X_test_normalized_v2[i]) + [y_test_v2[i]] for i in range(len(X_test_normalized_v2))]

# Set the learning rate and number of epochs
learning_rate = 0.0001
epochs = 1000

# # Example: Using the perceptron algorithm without cross-validation
accuracy_train, accuracy_test = evaluate_algorithm(train_dataset, test_dataset, perceptron, learning_rate, epochs)

# # Print the accuracy scores
print(f'Training Accuracy: {accuracy_train:.2f}%')
print(f'Testing Accuracy: {accuracy_test:.2f}%')

Training Accuracy: 77.39%
Testing Accuracy: 90.63%


# Notes

Yang digunakan itu yg Use 2 ya pada setiap model.
bedanya model 1 dan 2 itu almost neglible.